# Manual Neural Network
In this notebook we will manually build out a neural network that mimics the TensorFlow API. This will greatly help your understanding when working with the real TensorFlow!

________
### Quick Note on Super() and OOP

In [184]:
class SimpleClass:
    
    def __init__(self,name):
        print("Hello "+name)
        
    def yell(self):
        print("YELLING")

In [176]:
x = SimpleClass()

Hello 


In [177]:
x.yell()

YELLING


In [185]:
class ExtendedClass(SimpleClass):
    
    def __init__(self):
        super().__init__("Ronak")
        print("Extended")

In [186]:
y = ExtendedClass()

Hello Ronak
Extended


In [187]:
y.yell()

YELLING


________

## Operation

In [198]:
class Operation():
    """
    An Operation is a node in a "Graph". TensorFlow will also use this concept of a Graph.
    
    This Operation class will be inherited by other classes that actually compute the specific
    operation, such as adding or matrix multiplication.
    """
    
    def __init__(self,input_nodes=[]):
        """
        Intialize an Operation
        """
        self.input_nodes = input_nodes # The list of input nodes
        self.output_nodes = [] # List of nodes consuming this node's output
        
        # For every node in the input, we append this operation (self) to the list of
        # the consumers of the input nodes
        
        for node in input_nodes:
            node.output_nodes.append(self)
            
        # There will be a global default graph (TensorFlow works this way)
        # We will then append this particular operation
        # Append this operation to the list of operations in the currently active default graph
            
        _default_graph.operations.append(self)
        
    def compute(self):
        """ 
        This is a placeholder function. It will be overwritten by the actual specific operation
        that inherits from this class.
        
        """
        pass

## Example Operations

### Addition

In [199]:
class add(Operation):
    
    def __init__(self,x,y):
        
        super().__init__([x,y])
        
    def compute(self,x_var,y_var):
        
        self.inputs = [x_var,y_var]
        return x_var + y_var 

### Mutiplication

In [200]:
class multiply(Operation):
    
    def __init__(self,x,y):
        
        super().__init__([x,y])
        
    def compute(self,x_var,y_var):
        
        self.inputs = [x_var,y_var]
        return x_var * y_var 

### Matrix Multiplication

In [201]:
class matmul(Operation):
    
    def __init__(self,x,y):
        
        super().__init__([x,y])
        
    def compute(self,x_var,y_var):
        
        self.inputs = [x_var,y_var]
        return x_var.dot(y_var) 

## Placeholders

In [202]:
class Placeholder():
    """
    A placeholder is a node that needs to be provided a value for computing the output in the Graph.
    """
    
    def __init__(self):
        
        self.output_nodes = []
        _default_graph.placeholders.append(self)

## Variables

In [203]:
class Variable():
    """
    This variable is a changeable parameter of the Graph.
    """
    def __init__(self,initial_value=None):
        
        self.value = initial_value
        self.output_nodes = []
        _default_graph.variables.append(self)

## Graph

In [204]:
class Graph():
    
    def __init__(self):
        
        self.placeholders = []
        self.variables = []
        self.operations = []
        
    def set_as_default(self):
        """
        Sets this Graph instance as the Global Default Graph
        """
        global _default_graph
        _default_graph = self

# A Basic Graph
$$z = Ax + b$$

with A = 10  and b = 1

$$z = 10x + 1$$
Just need a placeholder for x and then once x is filled in we can solve it!

In [205]:
g = Graph()

In [206]:
g.set_as_default()

In [207]:
A = Variable(10)

In [208]:
b = Variable(1)

In [209]:
x = Placeholder()

In [210]:
y = multiply(A,x)

In [211]:
z = add(y,b)

# Session

In [213]:
import numpy as np

### Traversing Operation Nodes

In [215]:
def traverse_postorder(operation):
    """ 
    PostOrder Traversal of Nodes. Basically makes sure computations are done in 
    the correct order (Ax first , then Ax + b). Feel free to copy and paste this code.
    It is not super important for understanding the basic fundamentals of deep learning.
    """
    
    nodes_postorder = []
    def recurse(node):
        if isinstance(node,Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)
    recurse(operation)
    return nodes_postorder

In [216]:
class Session:
    
    def run(self, operation, feed_dict = {}):
        """ 
          operation: The operation to compute
          feed_dict: Dictionary mapping placeholders to input values (the data)  
        """
        
        # Puts nodes in correct order
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:

            if type(node) == Placeholder:
                
                node.output = feed_dict[node]
                
            elif type(node) == Variable:
                
                node.output = node.value
                
            else: # Operation
                
                node.inputs = [input_node.output for input_node in node.input_nodes]

                 
                node.output = node.compute(*node.inputs)
                
            # Convert lists to numpy arrays
            if type(node.output) == list:
                node.output = np.array(node.output)
        
        # Return the requested node value
        return operation.output


In [152]:
sess = Session()

In [153]:
result = sess.run(operation = z,feed_dict={x:10})

In [154]:
result

101

In [155]:
g = Graph()
g.set_as_default()
A = Variable([[10,20],[30,40]])
b = Variable([1,1])
x = Placeholder()
y = matmul(A,x)
z = add(y,b)

In [156]:
ses = Session()

In [157]:
ses.run(operation=z,feed_dict={x:10})

array([[101, 201],
       [301, 401]])

# Great Job!